This notebooks aims at exploring the dataset "bilan-electrique-demi-heure" from enedis

# Import packages

In [1]:
%run ./utils.ipynb

In [2]:
import requests
import pandas as pd
import os

# Import data

Request

In [3]:
if not(os.path.isfile("data/bilan-electrique.csv")):
#rewrite with pathlib
    print("Downloading data...")
    params = {
        "dataset" : "bilan-electrique-demi-heure",
        "rows" : 10000,
        "sort" : "horodate"
    }

    url = "https://data.enedis.fr/api/records/1.0/search/"

    req = requests.get(url, params=params).json() # http request on open data

In [4]:
if not(os.path.isfile("data/bilan-electrique.csv")):
    df = pd.json_normalize(req, record_path=["records"])
    column_rename_dict = {name : name[7:] for name in df.columns if name[:6] == "fields"}
    df.rename(columns=column_rename_dict, inplace=True)
    df.to_csv("data/bilan-electrique.csv")

In [5]:
df = pd.read_csv("data/bilan-electrique.csv")

In [6]:
df.head()

,Unnamed: 0,datasetid,recordid,record_timestamp,injection_rte,pertes,consommation_hta,soutirage_rte,production_profilee_photovoltaique,temperature_normale_lissee,...,consommation_profilee,production_profilee_aut,consommation_totale,mois,consommation_profilee_ent_hta,pseudo_rayonnement,soutirage_vers_autres_grd,horodate,consommation_telerelevee,production_photovoltaique
0,0,bilan-electrique-demi-heure,58084cf1db47bd052687d2808114953d19755216,2023-05-16T01:00:01.115Z,3.182218e+10,2.099434e+09,9.991167e+09,555480000.0,4304.0,14.3,...,1.851374e+10,1745767.0,3.177138e+10,5,9456817.0,57,290227009.0,2023-05-05T21:30:00+00:00,1.325764e+10,2060926.0
1,1,bilan-electrique-demi-heure,a0f3d1fa216f68beb1ba355e92718bb6582e7f88,2023-05-16T01:00:01.115Z,3.230077e+10,2.125454e+09,1.013050e+10,570125000.0,4608.0,14.3,...,1.866496e+10,1745767.0,3.219705e+10,5,9779855.0,57,288823000.0,2023-05-05T21:00:00+00:00,1.353209e+10,2153851.0
2,2,bilan-electrique-demi-heure,4abd9b6184d9c90f8ec85e3558fcac979e893297,2023-05-16T01:00:01.115Z,3.280950e+10,2.154360e+09,1.025865e+10,537333000.0,10363.0,14.4,...,1.884364e+10,1745767.0,3.264346e+10,5,10073269.0,56,282280700.0,2023-05-05T20:30:00+00:00,1.379982e+10,2176151.0
3,3,bilan-electrique-demi-heure,07382d88805d5a4ee9588fd6ff27c57af3630296,2023-05-16T01:00:01.115Z,3.122503e+10,2.003898e+09,1.036766e+10,477538000.0,10724.0,14.6,...,1.709235e+10,1745767.0,3.114739e+10,5,10514334.0,55,247365075.0,2023-05-05T20:00:00+00:00,1.405504e+10,2423144.0
4,4,bilan-electrique-demi-heure,ccf92afe83d1d3540f6cec6e79a574fd41992624,2023-05-16T01:00:01.115Z,3.107709e+10,1.987082e+09,1.038233e+10,452558000.0,7828.0,14.7,...,1.689479e+10,1745767.0,3.097478e+10,5,9587642.0,54,247766841.0,2023-05-05T19:30:00+00:00,1.407998e+10,2581626.0


Preprocess data

In [7]:
columns = ["horodate", "consommation_hta"]

In [8]:
df = df[columns]

df["horodate"] = pd.to_datetime(df["horodate"].apply(lambda x : x[:19]),
               format='%Y-%m-%dT%H:%M:%S')

/var/folders/q9/031bbh9935n746cd0kcr5msh0000gn/T/ipykernel_15052/1542010821.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["horodate"] = pd.to_datetime(df["horodate"].apply(lambda x : x[:19]),


# Data exploration

Plot time serie

In [9]:
fig = px.line(
    df,
    x="horodate",
    y="consommation_hta",
    title='HTA consumption'
    )
fig.show()

Group data at day level

In [10]:
df["date"] = df["horodate"].dt.date
df_day = df.groupby("date")["consommation_hta"].sum().reset_index()

/var/folders/q9/031bbh9935n746cd0kcr5msh0000gn/T/ipykernel_15052/2961940164.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
fig = px.line(
    df_day,
    x="date",
    y="consommation_hta",
    title='HTA consumption'
    )
fig.show()

Plot correlation diagrams

In [12]:
create_corr_plot(df_day["consommation_hta"])

In [13]:
create_corr_plot(df_day["consommation_hta"], plot_pacf=True)